<a href="https://colab.research.google.com/github/faiz-aljohani/ICS483-Project/blob/main/ICS483_PP2_EnlightenGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
!pip install wget

!git clone https://github.com/VITA-Group/EnlightenGAN.git
!mv /content/EnlightenGAN/* /content
!rm -r /content/EnlightenGAN

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=06eab4372d8d1826d78f3cf1781fdec4e1c5994bf400200d1e9aed9367cbac17
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Cloning into 'EnlightenGAN'...
remote: Enumerating objects: 603, done.
remote: Counting objects: 100% (417/417), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 603 (delta 388), reused 374 (delta 370), pack-reused 186
Receiving objects: 100% (603/603), 17.88 MiB | 13.25 MiB/s, done.
Resolving deltas: 100% (442/442), done.


In [ ]:
!gdown 1dzuLCk9_gE2bFF222n3-7GVUlSVHpMYC
!unzip LOL-v2.zip

In [ ]:
!gdown 1AkV-n2MdyfuZTFvcon8Z4leyVb0i7x63

if not os.path.exists("./checkpoints/enlightening"):
    os.makedirs("./checkpoints/enlightening")

!mv /content/200_net_G_A.pth ./checkpoints/enlightening

Downloading...
From: https://drive.google.com/uc?id=1AkV-n2MdyfuZTFvcon8Z4leyVb0i7x63
To: /content/200_net_G_A.pth
100% 34.6M/34.6M [00:00<00:00, 154MB/s]


In [ ]:
!pip install dominate
!pip install visdom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=04e809aa13382d717faeb480a62af68155b0561001056313256c8cd3c9bdc33d
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [ ]:
if not os.path.exists("../test_dataset/testA"):
    os.makedirs("../test_dataset/testA")

if not os.path.exists("../test_dataset/testB"):
    os.makedirs("../test_dataset/testB")

!cp /content/LOL-v2/Real_captured/Test/Low/* ../test_dataset/testA
!cp ../test_dataset/testA/low00690.png ../test_dataset/testB

In [ ]:
!python scripts/script.py --predict

------------ Options -------------
D_P_times2: False
IN_vgg: False
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
dataroot: ../test_dataset
dataset_mode: unaligned
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fcn: 0
fineSize: 256
gpu_ids: [0]
high_times: 400
how_many: 50
hybrid_loss: False
identity: 0.0
input_linear: False
input_nc: 3
instance_norm: 0.0
isTrain: False
l1: 10.0
lambda_A: 10.0
lambda_B: 10.0
latent_norm: False
latent_threshold: False
lighten: False
linear: False
linear_add: False
loadSize: 286
low_times: 200
max_dataset_size: inf
model: single
multiply: False
nThreads: 4
n_layers_D: 3
n_layers_patchD: 3
name: enlightening
ndf: 64
new_lr: False
ngf: 64
no_dropout: True
no_flip: False
no_vgg_instance: False
noise: 0
norm: instance
norm_attention: False
ntest: inf
output_nc: 3
patchD: False
patchD_3: 0
patchSize: 64
patch_vgg: False
phase: test
resize_or_crop: no
results_dir: ./results/
self_attention: True
serial_batches

In [ ]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.8 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision.transforms as T
import os
from torcheval.metrics import PeakSignalNoiseRatio, MeanSquaredError
from torchvision.transforms.functional import to_tensor
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import re
import cv2
import numpy as np

def prepare_data(enhanced_dir_path, groundtruth_dir_path):
    # Custom folder preparation and filtering
    enhanced_images = sorted(os.listdir(enhanced_dir_path))
    enhanced_images = [enhanced_dir_path + "/" +imgName for imgName in enhanced_images if re.match(r'.*real_A\.png$', imgName)]
    groundtruth_images = sorted(os.listdir(groundtruth_dir_path))
    groundtruth_images = [groundtruth_dir_path + "/" + imgName for imgName in groundtruth_images]
    return enhanced_images, groundtruth_images

def evaluate_psnr(enhanced_images, groundtruth_images):
    transform = T.ToTensor()
    metric = PeakSignalNoiseRatio()
    eval_list = []

    for i in range(len(enhanced_images)):
        enhanced_img = Image.open(enhanced_images[i])
        groundtruth_img = Image.open(groundtruth_images[i])

        tensor_enhanced_img = transform(enhanced_img)
        tensor_groundtruth_img = transform(groundtruth_img)

        metric.update(tensor_enhanced_img, tensor_groundtruth_img)
        eval_list.append(metric.compute().item())

    result = sum(eval_list) / len(eval_list)
    return result


def evaluate_ssim(enhanced_images, groundtruth_images):
  eval_list = []

  for i in range(len(enhanced_images)):
    enhanced_img = Image.open(enhanced_images[i])
    groundtruth_img = Image.open(groundtruth_images[i])

    enhanced_np = np.array(enhanced_img)
    groundtruth_np = np.array(groundtruth_img)

    ssim_for_each_channel = []
    for channel in range(3):
      ssim_score_channel = ssim(enhanced_np[:, :, channel], groundtruth_np[:, :, channel])
      ssim_for_each_channel.append(ssim_score_channel)

    avg_ssim_channels = np.mean(ssim_for_each_channel)
    eval_list.append(avg_ssim_channels)

  result = np.mean(eval_list)
  return result


def evaluate_mse(enhanced_images, groundtruth_images):
  eval_list = []

  for i in range(len(enhanced_images)):
    enhanced_img = np.array(Image.open(enhanced_images[i]))
    groundtruth_img = np.array(Image.open(groundtruth_images[i]))

    mse = np.mean(np.square(enhanced_img - groundtruth_img))
    eval_list.append(mse)

  result = np.mean(eval_list)
  return result


def evaluate_methods(enhanced_dir_path, groundtruth_dir_path, method):
    enhanced_images, groundtruth_images = prepare_data(enhanced_dir_path, groundtruth_dir_path)
    if method == "PSNR":
        result = evaluate_psnr(enhanced_images, groundtruth_images)
    elif method == "SSIM":
        result = evaluate_ssim(enhanced_images, groundtruth_images)
    elif method == "MSE":
        result = evaluate_mse(enhanced_images, groundtruth_images)
    else:
        raise ValueError("Unsupported evaluation method.")

    return result

In [ ]:
enhanced_dir_path = "/content/ablation/enlightening/test_200/images"
groundtruth_dir_path = "/content/LOL-v2/Real_captured/Test/Normal"

methods = ["PSNR","SSIM","MSE"]
for method in methods:
    result = evaluate_methods(enhanced_dir_path, groundtruth_dir_path, method)
    print("The {} for the test images is: {:.2f}".format(method, result))

The PSNR for the test images is: 8.45
The SSIM for the test images is: 0.18
The MSE for the test images is: 109.66
